In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from model.model_fast import City

In [ ]:
num_steps  = 200
parameters = {
            'width':     3,
            'height':    3,
}
city = City(num_steps, **parameters)
for time_step in range(num_steps): # TODO time here and above
    # print(f'Step {time}')
    city.step_fast()
city.record_run_data_to_file()
agent_out = city.datacollector.get_agent_vars_dataframe()
model_out = city.datacollector.get_model_vars_dataframe()

In [ ]:
# Extract attributes for plotting (excluding 'dist')
attributes_to_plot = ["R_N", "p_dot", "investor_bid", "warranted_rent", "warranted_price", "maintenance"]

# Create subplots
num_rows = 4
num_cols = 2
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))
fig.suptitle("Attributes vs Distance at Each Time Step", y=1.02, fontsize='x-large')  # Increased font size for the title

# Get distinct colors for each time step
num_time_steps = city.num_steps
colors = plt.cm.viridis(np.linspace(0, 1, num_time_steps))

# Loop over attributes
for i, attribute in enumerate(attributes_to_plot):
    # Loop over time steps
    for time_step in range(2, num_time_steps + 1):
        # Load data for the current time step
        loaded_data = model_out[model_out['time_step'] == time_step]

        # Flatten the 'dist' column if it contains lists
        flattened_dist = [item for sublist in loaded_data['dist'].values for item in sublist]

        # Plot the attribute on the current subplot with the same color for each time step
        axs[i // num_cols, i % num_cols].plot(
            np.array(flattened_dist).astype(float),
            loaded_data[attribute].explode(),
            label=f"Time Step {time_step}",
            color=colors[time_step - 1]
        )
        axs[i // num_cols, i % num_cols].set_title(attribute, fontsize='large')  # Increased font size for the title
        axs[i // num_cols, i % num_cols].set_xlabel("Distance", fontsize='medium')  # Increased font size for x-axis label
        axs[i // num_cols, i % num_cols].set_ylabel(attribute, fontsize='medium')  # Increased font size for y-axis label

# Add a legend to the last subplot
legend_ax = axs[-1, -1]
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, title='Time Steps', bbox_to_anchor=(.9, .10), loc='lower right', fontsize='large')
# legend_ax.legend(handles, labels, title='Time Steps', bbox_to_anchor=(1.05, 1), loc='upper left')
legend_ax.axis('off')  # Disable axis for the legend

plt.tight_layout()
plt.show()


In [ ]:
# Plot 
for time_step in range(1, num_steps + 1):
    # Load newcomer bid data for the current time step
    loaded_data = model_out[model_out['time_step'] == time_step]['newcomer_bid'].values[0]

    # Extract data
    newcomer_bids, dist_values, savings_values = zip(*loaded_data)

    # Create a 3D surface plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(dist_values, savings_values, newcomer_bids, cmap='viridis', edgecolor='k')
    ax.scatter(dist_values, savings_values, newcomer_bids, marker='*', color='red', s=100, label='Newcomer Bids')
    ax.set_title(f'Newcomer Bids - Time Step {time_step}')
    ax.set_xlabel('Distance')
    ax.set_ylabel('Savings')
    ax.set_zlabel('Newcomer Bid')
    plt.show()